In [1]:
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 32.1 MB/s 
     |████████████████████████████████| 1.6 MB 70.8 MB/s 


In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import datetime
from tqdm import tqdm

In [3]:
try:
  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')
except:
  path = '/content/drive/MyDrive/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')

# 11월 넘어가는 유저 제거

In [4]:
df['event_time'] = df['event_time'].apply(lambda x : x[:-4]).astype('datetime64')
df['event_time'].dtype

dtype('<M8[ns]')

In [5]:
df['event_time'] = df['event_time'] + datetime.timedelta(hours=4)
df = df.loc[df['event_time'] < '2019-11-01']

In [6]:
df.tail()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
42398925,2019-10-31 23:59:59,view,12718908,452985123,NaN,yokohama,96.269997,536689953,0a77064e-0bc6-4745-b703-5e8208bf8f6e
42398926,2019-10-31 23:59:59,view,28719204,-352320551,apparel.shoes,alpina,220.080002,534245704,699164ab-a8f6-4ce0-bd07-b059ff3e1db2
42398927,2019-10-31 23:59:59,view,1005217,-1769995873,electronics.smartphone,xiaomi,151.070007,560095437,15e037ed-4f14-44ba-b7c0-0c033a6c3649
42398928,2019-10-31 23:59:59,view,1003892,-1769995873,electronics.smartphone,xiaomi,144.729996,566243306,566c6d0a-2cd5-4248-ac7b-3af675c05fd3
42398929,2019-10-31 23:59:59,view,17100010,947913283,NaN,NaN,77.199997,512824822,dee78bdc-718a-42bd-8b78-a26259cd1078


## cart, purchase가 존재하는 유저 데이터 추출

1. event_type 컬럼에 cart가 존재하는 user_id 추출
2. event_type 컬럼에 purchase가 존재하는 user_id 추출
3. 위에서 구한 id 합치기

In [8]:
cart_user_id = set(df.loc[df['event_type'] == 'cart','user_id'].unique())
purchase_user_id = set(df.loc[df['event_type'] == 'purchase', 'user_id'].unique())

all_id = cart_user_id.union(purchase_user_id)

In [9]:
df = df.loc[df['user_id'].isin(all_id)].reset_index(drop=True)

In [10]:
df['event_type'] = df['event_type'].astype('object')

# 데이터 처리

In [19]:
df.sample(5)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
6743077,2019-10-12 23:30:29,view,5100826,268435735,NaN,ritmix,13.640000,513846960,034c46a9-c9f1-4fdb-86b4-24b3e7a9092c
16078281,2019-10-27 09:45:21,cart,5100564,234881301,electronics.clocks,samsung,257.869995,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
9655674,2019-10-16 23:48:50,view,33200001,-243268902,NaN,joerex,10.300000,514996695,205d56bd-1082-f6aa-ee42-96b88a0c9bda
12203588,2019-10-20 20:36:09,view,17302526,746586423,NaN,afnan,24.000000,512364192,8b36e62e-c570-45ca-a887-22038c59fd05
12607661,2019-10-21 14:18:44,view,26500140,1853883237,NaN,lucente,174.259995,513103843,cd1d5d2e-dc9c-4067-819c-386faa898ecb


In [20]:
df.loc[df['user_id'] == 517236578]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
16076044,2019-10-27 09:41:27,view,5100719,234881301,electronics.clocks,samsung,151.610001,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16076297,2019-10-27 09:41:54,view,5100564,234881301,electronics.clocks,samsung,257.869995,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16077928,2019-10-27 09:44:43,view,5100564,234881301,electronics.clocks,samsung,257.869995,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16078176,2019-10-27 09:45:10,cart,5100564,234881301,electronics.clocks,samsung,257.869995,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16078215,2019-10-27 09:45:14,view,5100564,234881301,electronics.clocks,samsung,257.869995,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16078281,2019-10-27 09:45:21,cart,5100564,234881301,electronics.clocks,samsung,257.869995,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16080067,2019-10-27 09:48:31,view,5100337,234881301,electronics.clocks,apple,318.239990,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16081202,2019-10-27 09:50:38,view,1004249,-1769995873,electronics.smartphone,apple,725.080017,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16081470,2019-10-27 09:51:05,view,1004838,-1769995873,electronics.smartphone,oppo,141.279999,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
16081838,2019-10-27 09:51:43,view,1005160,-1769995873,electronics.smartphone,xiaomi,215.130005,517236578,f9fb2fde-2e83-4d1b-8ab0-f16fd7a74e1f
